In [1]:
from transformers import Wav2Vec2Processor, HubertForCTC
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import soundfile as sf
import torch
from time import time
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [26]:
import os
files = os.listdir('DTLN_DPTNET_FLAC/')

In [27]:
minimum = 9999999
with torch.no_grad():
    liste_denoised = []
    for file in files:
        print(file)
        ds = sf.read('DTLN_DPTNET_FLAC/'+file)
        input_values = processor(ds[0], return_tensors="pt", padding="longest",num_process=6,sampling_rate=16000).input_values  # Batch size 1
        output =  model(input_values)
        output = torch.argmax(output.logits, dim=-1)
        output = output.detach().to(device='cpu', dtype=torch.long)
        transcription = processor.batch_decode(output)
        print(transcription)
        liste_denoised.append(transcription)

p226_010.flac
['PEOPLE LOOK BUT NO ONE EVER FINDS IT']
p226_006.flac
['WHEN THE SUNLIGHT STRIKES RAINDROPS IN THE AIR THEY ACT AS A PRISOM AND FORM A RAINBOW']
p226_007.flac
['THE RAINBOW IS A DIVISION OF WHITE LIGHT INTO MANY BEAUTIFUL COLORS']
p226_001.flac
['NOW PLEASE CALL STELLA']
p226_003.flac
['THAT BEAG SIX SPOONS AND BREAFAST NOTE AN UNTIL IT ARRIVED FIVE THIX FLOWVERY AT MUCHI LE AND MAYBE A TSNAP BY HER BROTHER FOLLOW HER L']
p226_009.flac
['THERE IS ACCORDING TO A LEGEND A BOILING POT OF GOLD AT ONE END']
p226_004.flac
['NO OT WE OUGH TO NEED AN NO A BATTER NET THE HAR ADOL AND OUR BIG TOY FOR OIL IN MY CASE TOM  YOU']
p226_005.flac
['SHE CAN SCOOP THESE THINGS INTO THREE RED BAGS AND WE WILL GO MEET HER WEDNESDAY AT THE TRAIN STATION']
p226_002.flac
['ASK HER TO BRING THESE THINGS WITH HER IN THE STORY ORWHAT']
p226_008.flac
['SE TAKE THE SHAPE OF THE LONG ROUND ARCH WITH ITS PATH HIGH ABOVE I AND ITS TWO ENDS APPARENTLY BEYOND THE HORIZON']


In [28]:
minimum = 9999999
with torch.no_grad():
    liste_clean = []
    for file in files:
        print(file)
        ds = sf.read('clean_trainset_28spk_wav_FLAC/'+file)
        input_values = processor(ds[0], return_tensors="pt", padding="longest",num_process=6,sampling_rate=16000).input_values  # Batch size 1
        output =  model(input_values)
        output = torch.argmax(output.logits, dim=-1)
        output = output.detach().to(device='cpu', dtype=torch.long)
        transcription = processor.batch_decode(output)
        print(transcription)
        liste_clean.append(transcription)

p226_010.flac
['PEOPLE LOOK BUT NO ONE EVER FINDS IT']
p226_006.flac
['WHEN THE SUNLIGHT STRIKES RAINDROPS IN THE AIR THEY ACT AS A PRISM AND FORM A RAINBOW']
p226_007.flac
['THE RAINBOW IS A DIVISION OF WHITE LIGHT INTO MANY BEAUTIFUL COLOURS']
p226_001.flac
['PLEASE CALL STELLA']
p226_003.flac
['SIX SPOONS OF FRESH SNOW PEASE FIVE THICK SLABS OF BLUE CHEESE AND MAYBE A SNACK FOR HER BROTHER BOB']
p226_009.flac
['THERE IS ACCORDING TO A LEGEND A BOILING POT OF GOLD AT ONE END']
p226_004.flac
['WE ALSO NEED A SMALL PLASTIC SNAKE AND A BIG TOY FROG FOR THE KIDS']
p226_005.flac
['SHE CAN SCOOP THESE THINGS INTO THREE RED BAGS AND WE WILL GO MEET HER WEDNESDAY AT THE TRAIN STATION']
p226_002.flac
['ASK HER TO BRING THESE THINGS WITH HER FROM THE STORE']
p226_008.flac
['THESE TAKE THE SHAPE OF A LONG ROUND ARCH WITH ITS PATH HIGH ABOVE AND ITS TWO ENDS APPARENTLY BEYOND THE HORIZON']


In [29]:
import numpy

def get_word_error_rate(r, h):

    """
    Given two list of strings how many word error rate(insert, delete or substitution).
    """
    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint16)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    result = float(d[len(r)][len(h)]) / len(r) * 100

    return result


In [30]:
liste_results = []
for i in range(len(liste_clean)):
    liste_results.append(get_word_error_rate(liste_denoised[i][0].split(),liste_clean[i][0].split()))

In [31]:
liste_results

[0.0,
 5.88235294117647,
 8.333333333333332,
 25.0,
 70.37037037037037,
 0.0,
 76.0,
 0.0,
 25.0,
 13.043478260869565]

In [32]:
np.mean(liste_results)

22.362953490574974

In [22]:
#!/usr/bin/env python


def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy

    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]

In [23]:
liste_results = []
for i in range(len(liste_clean)):
    liste_results.append(wer(liste_denoised[i][0].split(),liste_clean[i][0].split()))

In [24]:
np.mean(liste_results)


4.7